In [1]:
import pandas as pd 

In [4]:
cross_play_runs = '../../data/processed_outputs/cross_play/16-11-2023_19-45-16/evals.csv'
self_play_runs = '../../data/processed_outputs/self_play/29-10-2023_11-21-36/evals.csv'

In [5]:
cross_play = pd.read_csv(cross_play_runs)
self_play = pd.read_csv(self_play_runs)

In [6]:
df = pd.concat([cross_play, self_play])

In [7]:
def error_present(dff):
    # TODO: this may be problematic if they think they have interests opposite to what they have.
    # TODO: should it be full agreement or synthetic?
    df = dff.copy()
    df['error'] = False 
    df.loc[((df['agent_1_normalized_payoff'] == 0) & (
            df['agent_2_normalized_payoff'] == 0) & (df["completion_reason"] == 'full agreement')), 'error'] = True 
    df.loc[((df['agent_1_normalized_payoff'] != 1 - df['agent_2_normalized_payoff']) & (
                df["completion_reason"] == 'full agreement') & (df['game_type'] == 'non-integrative distributive')), 'error'] = True 
    df.loc[((df['agent_1_normalized_payoff'] != df['agent_2_normalized_payoff']) & (
            df["completion_reason"] == 'full agreement') & (df['game_type'] == 'non-integrative compatible') &
              (df['num_issues'] == 1)), 'error'] = True 
    return df

In [8]:
df = error_present(df)

In [9]:
df['synthetic_agreement'] = df['completion_reason'].apply(lambda x: 1 if x in ['aligning internal states, textual disagreement', 'full agreement'] else 0)
df['full_agreement'] = df['completion_reason'].apply(lambda x: 1 if x in ['full agreement'] else 0)

In [10]:
df.groupby(['agent_1_model_name', 'agent_2_model_name', 'num_issues', 'game_type', 'protocol_start_agent_index']).agg({'protocol_start_agent_index': 'count', 'synthetic_agreement': 'sum', 'full_agreement': 'sum', 'error': 'sum'}).rename({'protocol_start_agent_index': 'count'}, axis=1).to_csv('../../data/processed_outputs/summary_of_all_runs.csv')